In [ ]:
from gpsprocessor import gps_data_utils
from shapely.geometry import Point, LineString
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

### Set Depot Points, Boundary and Buffer here

In [ ]:
p1 = Point(19.421231015290946, -99.12768416729142)
p2 = Point(19.420298241433308, -99.12729008610742)
p3 = Point(19.419116629726144, -99.12747041774)
p4 = Point(19.41926477082061, -99.12969297748691)
p5 = Point(19.42117868427646, -99.12944573801215)

points = [p1,p2,p3,p4,p5,p1]
depot = gps_data_utils.set_depot_boundary(points,buffer = 0)

### Read and Validate GPS Data here

In [ ]:
'''
error_factor here is used as 1.5, i.e., 
    relative_error = abs(velocity_gps-velocity_vehicle) / velocity_vehicle 
    =>  ( 5 - 1 ) / 1 > 1.5
    => Calculated GPS speed is way higher than observed speed of the vehicle, i.e. gps walking
    => higher error_factor will allow more GPS walking
'''
error_factor = 1.25

In [ ]:
cols = ['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus']
date_col = ['RowReferenceTime']
d_format = 'ISO8601'

file = 'D:/Extra-Projects/MGL-India/Moving Dots/trip_data_10Units_1Year_V2.csv'
df = gps_data_utils.read_data(file,cols,date_col,d_format)

In [ ]:
df = gps_data_utils.validate_mandatory_cols(df,'RowReferenceTime','lat','lon','Speed','UnityLicensePlate')

In [ ]:
df = gps_data_utils.add_date_time_month_df(df)

In [ ]:
df = df.groupby(by=['vehicleid','date']).apply(gps_data_utils.add_prev_latlong).reset_index(drop=True)

In [ ]:
df = gps_data_utils.check_veh_within_depot(df,depot)

In [ ]:
df = gps_data_utils.calculate_speed_flag(df, error_factor)

In [ ]:
df = gps_data_utils.blank_filter(df)

In [ ]:
gdf = gps_data_utils.df_to_gdf(df)
tolerance = 0.05

gdf_grouped = gdf.groupby(by=['vehicleid'])

line_gdf = []

for vehicleid, group in gdf_grouped:

    line = LineString(zip(group['lon'], group['lat']))
    
    line_gdf.append(gpd.GeoDataFrame(geometry=line,crs='EPSG:4326'))

# Optionally, you can set the coordinate reference system (CRS) for the GeoDataFrame


# simplified_lines = gdf['geometry'].simplify(tolerance)
# simplified_gdf = gpd.GeoDataFrame(geometry=simplified_lines)

In [ ]:
maps = folium.Map(max_zoom=24)
fg = MarkerCluster().add_to(maps)
for _, row in simplified_gdf.iterrows():
    lon, lat = row['geometry'].x, row['geometry'].y
    folium.Marker(location=[lat, lon],popup=f"Index:{row.name}").add_to(fg)

In [ ]:
maps.fit_bounds(fg.get_bounds())
maps